In [ ]:
# %%capture
# !pip install -U --pre --extra-index https://pypi.anaconda.org/scipy-wheels-nightly/simple scikit-learn

In [3]:
import cv2
# import notebook26452d67f4 as viz_utils
from fastai.vision.all import *
from sklearn import model_selection
from tqdm.auto import tqdm
import seaborn as sns
tqdm.pandas()

## Albumentations format

In [4]:
best_sub = pd.read_csv('../input/wbf-ensemble-v3-0/yolov5_effdet5_iou0.6.csv')

In [5]:
best_sub.head()

,image_id,PredictionString
0,0026720152f5_image,opacity 0.3270 450 1106 1137 2241 opacity 0.2562 1832 1283 2447 2296 opacity 0.2326 440 1655 1094 2273 opacity 0.1483 1744 1654 2392 2350 opacity 0.1377 2130 1875 2415 2276 opacity 0.1085 1927 1811 2417 2322 opacity 0.1019 2067 1552 2444 2297 opacity 0.0907 420 1406 878 2228 opacity 0.0709 1666 894 2461 2323 opacity 0.0659 535 1132 977 2019 opacity 0.0640 651 1393 1162 2176 opacity 0.0623 552 1133 904 1509 opacity 0.0561 473 1785 889 2213 opacity 0.0517 530 1000 975 1598 opacity 0.0500 1669 1437 2220 2282 opacity 0.0479 1778 1854 2229 2303 opacity 0.0477 2190 1925 2377 2160 opacity 0.0471 ...
1,00fc8fc35dc1_image,opacity 0.5071 2327 898 3281 2438 opacity 0.4531 608 591 1608 2330 opacity 0.1343 701 728 1476 1723 opacity 0.1051 613 1333 1457 2324 opacity 0.1002 596 853 1245 2160 opacity 0.0886 2692 1071 3286 2357 opacity 0.0681 2047 299 3349 2692 opacity 0.0644 646 1608 1579 2360 opacity 0.0513 2432 1292 3128 2319 opacity 0.0456 687 790 1173 1567 opacity 0.0311 402 196 1842 2741 opacity 0.0310 794 980 1529 2145 opacity 0.0277 2244 1412 3363 2698 opacity 0.0266 729 1639 1337 2220 opacity 0.0265 511 1283 1622 2648 opacity 0.0245 975 963 1661 2309 opacity 0.0235 2597 871 3178 2008 opacity 0.0234 2499 13...
2,01c3512eebc3_image,opacity 0.2045 424 1331 1068 1943 opacity 0.2034 1764 1482 2360 2009 opacity 0.1641 1987 1604 2366 1983 opacity 0.0945 412 726 1102 1910 opacity 0.0908 1696 828 2324 1932 opacity 0.0876 1883 1093 2368 1956 opacity 0.0841 1681 1415 2182 1941 opacity 0.0827 363 1493 868 1958 opacity 0.0714 358 1023 874 1912 opacity 0.0674 2038 1437 2426 2048 opacity 0.0620 2006 1256 2386 1948 opacity 0.0570 308 1431 703 1998 opacity 0.0564 692 1179 1136 1823 opacity 0.0544 864 531 1180 994 opacity 0.0528 324 1595 648 1998 opacity 0.0525 733 1449 1119 1845 opacity 0.0524 336 1278 807 1982 opacity 0.0503 1658 ...
3,01f948f8e544_image,opacity 0.2626 577 1071 1161 1870 opacity 0.2141 1885 907 2410 1810 opacity 0.1107 604 1021 1018 1603 opacity 0.0819 1791 837 2309 1545 opacity 0.0786 755 1207 1257 1872 opacity 0.0766 2004 1052 2477 1965 opacity 0.0721 733 1423 1190 1891 opacity 0.0703 488 1017 977 1895 opacity 0.0667 587 981 835 1361 opacity 0.0664 1681 598 2459 1903 opacity 0.0617 551 642 1293 1930 opacity 0.0607 1709 1035 2284 1913 opacity 0.0605 2065 920 2424 1620 opacity 0.0569 552 1446 1113 1943 opacity 0.0550 2114 1083 2433 1726 opacity 0.0534 1971 1553 2392 1982 opacity 0.0495 1845 1317 2414 2003 opacity 0.0470 21...
4,022146012034_image,opacity 0.3526 386 1323 1054 1906 opacity 0.2714 1905 1467 2460 2098 opacity 0.0781 1797 956 2506 2128 opacity 0.0694 356 820 1104 1935 opacity 0.0670 535 1370 968 1806 opacity 0.0643 609 1295 1166 1886 opacity 0.0620 1733 1352 2316 2072 opacity 0.0535 1984 1582 2350 2053 opacity 0.0436 281 1240 853 1944 opacity 0.0432 1974 1124 2476 1989 opacity 0.0402 228 1215 1201 2094 opacity 0.0395 2036 1590 2557 2210 opacity 0.0358 2071 1291 2575 2101 opacity 0.0335 1784 1607 2316 2117 opacity 0.0329 1726 1389 2609 2251 opacity 0.0311 2043 1410 2426 1924 opacity 0.0254 393 1055 941 1813 opacity 0.023...


In [6]:
meta = pd.read_csv('../input/siim-covid19-png-1024px/meta.csv')

In [8]:
meta['image_id'] = meta['image_id'] + '_image'

In [11]:
best_sub = best_sub.merge(meta[meta['split']=='test'],on='image_id')

In [22]:
best_sub['PredictionString'].apply(lambda x: x.split('opacity'))[0][1][8:]

'450 1106 1137 2241 '

In [38]:
    pred_locations = {}
    ids = []
    pred_loc = []
    # for image_id in range(len(best_sub['image_id'])):
    # load table
    with open('../input/wbf-ensemble-v3-0/yolov5_effdet5_iou0.6.csv') as infile:
        # open reader
        reader = csv.reader(infile)
        # skip header
        next(reader, None)
        # loop through rows
        for rows in reader:
            # retrieve information
            filename = rows[0]
            # print(rows[0])
            parts = rows[1].split()
            # print(parts)
            assert len(parts) % 6 == 0
            locations = []
            for ind in range(len(parts) // 6):
                #label = int(parts[ind * 6])
             #   label = parts[ind * 6]
             #   label = 1 if label == 'opacity' else 0
                score = float(parts[ind * 6 + 1])
                location = int(float(parts[ind * 6 + 2])), int(float(parts[ind * 6 + 3])), \
                           int(float(parts[ind * 6 + 4])), int(float(parts[ind * 6 + 5]))
                if score > 0.2:
                    locations.append((location))
            if len(locations)>0:
                ids.append(filename)
                pred_loc.append(locations)
            #if score > 0.2:
            #    pred_loc.append(locations)
            #if filename in pred_locations:
            #    pred_locations[filename].append(locations)
            #else:
            #    pred_locations[filename] = [locations]

In [39]:
pred_loc[0]

[(450, 1106, 1137, 2241), (1832, 1283, 2447, 2296), (440, 1655, 1094, 2273)]

In [40]:
len(ids)

1186

In [41]:
len(pred_loc)

1186

In [44]:
best_sub = best_sub[best_sub['image_id'].isin(ids)]

In [45]:
best_sub['boxes'] = pred_loc

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
best_sub = best_sub.drop(['PredictionString'],axis=1,inplace=False)

In [47]:
best_sub.loc[0,'boxes']

,image_id,dim0,dim1,split,boxes
0,0026720152f5_image,2996,2988,test,"[(450, 1106, 1137, 2241), (1832, 1283, 2447, 2296), (440, 1655, 1094, 2273)]"
1,00fc8fc35dc1_image,4240,3480,test,"[(2327, 898, 3281, 2438), (608, 591, 1608, 2330)]"
2,01c3512eebc3_image,2801,2802,test,"[(424, 1331, 1068, 1943), (1764, 1482, 2360, 2009)]"
3,01f948f8e544_image,2991,2992,test,"[(577, 1071, 1161, 1870), (1885, 907, 2410, 1810)]"
4,022146012034_image,2801,2802,test,"[(386, 1323, 1054, 1906), (1905, 1467, 2460, 2098)]"
...,...,...,...,...,...
1258,fed3a460884c_image,2330,2846,test,"[(1572, 733, 2118, 1675), (411, 562, 1082, 1478), (388, 950, 959, 1483)]"
1259,ff5448be90d5_image,3520,4280,test,"[(2933, 1110, 3554, 2418), (720, 1072, 1350, 2420)]"
1260,ff88ed1df29b_image,2336,2836,test,"[(1840, 1315, 2437, 1856)]"
1261,ffb8115a304c_image,2701,2990,test,"[(591, 548, 1240, 1655), (1855, 711, 2374, 1639)]"


In [48]:
def clamp(minimum, x, maximum): return max(minimum, min(x, maximum))

def get_norm_bb(row):
    boxes = row.boxes
    annots = []


    for box in boxes:
        x1,y1,x2,y2 = box
        x1,x2 = x1/row.dim1 , x2/row.dim1
        y1,y2 = y1/row.dim0 , y2/row.dim0
        x1, y1, x2, y2 = clamp(0, x1, 1), clamp(0, y1, 1), clamp(0, x2, 1), clamp(0, y2, 1)
        annots.append([x1,y1,x2,y2])
        
    return annots

easy_df = best_sub.copy()
easy_df['bbox'] = easy_df.progress_apply(lambda row: get_norm_bb(row), axis=1)
display_df(easy_df.head())

  0%|          | 0/1186 [00:00<?, ?it/s]

,image_id,dim0,dim1,split,boxes,bbox
0,0026720152f5_image,2996,2988,test,"[(450, 1106, 1137, 2241), (1832, 1283, 2447, 2296), (440, 1655, 1094, 2273)]","[[0.15060240963855423, 0.3691588785046729, 0.38052208835341367, 0.7479973297730307], [0.6131191432396251, 0.42823765020026705, 0.8189424364123159, 0.7663551401869159], [0.14725568942436412, 0.5524032042723631, 0.36613119143239625, 0.7586782376502003]]"
1,00fc8fc35dc1_image,4240,3480,test,"[(2327, 898, 3281, 2438), (608, 591, 1608, 2330)]","[[0.6686781609195402, 0.2117924528301887, 0.9428160919540229, 0.575], [0.17471264367816092, 0.1393867924528302, 0.46206896551724136, 0.5495283018867925]]"
2,01c3512eebc3_image,2801,2802,test,"[(424, 1331, 1068, 1943), (1764, 1482, 2360, 2009)]","[[0.15132048536759457, 0.4751874330596216, 0.3811563169164882, 0.6936808282756158], [0.6295503211991434, 0.5290967511602999, 0.8422555317630264, 0.7172438414851838]]"
3,01f948f8e544_image,2991,2992,test,"[(577, 1071, 1161, 1870), (1885, 907, 2410, 1810)]","[[0.1928475935828877, 0.35807422266800404, 0.3880347593582888, 0.6252089602139752], [0.6300133689839572, 0.303243062520896, 0.80548128342246, 0.6051487796723504]]"
4,022146012034_image,2801,2802,test,"[(386, 1323, 1054, 1906), (1905, 1467, 2460, 2098)]","[[0.1377587437544611, 0.4723313102463406, 0.3761598857958601, 0.6804712602641914], [0.6798715203426124, 0.523741520885398, 0.8779443254817987, 0.7490182077829347]]"


In [50]:
best_sub = easy_df[['image_id','bbox']]
best_sub['split'] = -1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [51]:
best_sub

,image_id,bbox,split
0,0026720152f5_image,"[[0.15060240963855423, 0.3691588785046729, 0.38052208835341367, 0.7479973297730307], [0.6131191432396251, 0.42823765020026705, 0.8189424364123159, 0.7663551401869159], [0.14725568942436412, 0.5524032042723631, 0.36613119143239625, 0.7586782376502003]]",-1
1,00fc8fc35dc1_image,"[[0.6686781609195402, 0.2117924528301887, 0.9428160919540229, 0.575], [0.17471264367816092, 0.1393867924528302, 0.46206896551724136, 0.5495283018867925]]",-1
2,01c3512eebc3_image,"[[0.15132048536759457, 0.4751874330596216, 0.3811563169164882, 0.6936808282756158], [0.6295503211991434, 0.5290967511602999, 0.8422555317630264, 0.7172438414851838]]",-1
3,01f948f8e544_image,"[[0.1928475935828877, 0.35807422266800404, 0.3880347593582888, 0.6252089602139752], [0.6300133689839572, 0.303243062520896, 0.80548128342246, 0.6051487796723504]]",-1
4,022146012034_image,"[[0.1377587437544611, 0.4723313102463406, 0.3761598857958601, 0.6804712602641914], [0.6798715203426124, 0.523741520885398, 0.8779443254817987, 0.7490182077829347]]",-1
...,...,...,...
1258,fed3a460884c_image,"[[0.5523541813070977, 0.31459227467811157, 0.7442023893183415, 0.7188841201716738], [0.1444132115249473, 0.24120171673819743, 0.38018271257905834, 0.6343347639484979], [0.13633169360505973, 0.40772532188841204, 0.336964160224877, 0.636480686695279]]",-1
1259,ff5448be90d5_image,"[[0.6852803738317756, 0.3153409090909091, 0.8303738317757009, 0.6869318181818181], [0.16822429906542055, 0.30454545454545456, 0.31542056074766356, 0.6875]]",-1
1260,ff88ed1df29b_image,"[[0.6488011283497884, 0.5629280821917808, 0.859308885754584, 0.7945205479452054]]",-1
1261,ffb8115a304c_image,"[[0.19765886287625417, 0.2028878193261755, 0.41471571906354515, 0.6127360236949279], [0.6204013377926422, 0.26323583857830435, 0.7939799331103679, 0.6068122917437986]]",-1


In [52]:
best_sub.to_csv("grouped_df.csv", index=False)

In [ ]:
# !ls -l

In [ ]:
# !mkdir csv
# !mv grouped_df.csv csv
# !mv train_bbox_data.csv csv
# !mkdir coco

# !mv train_annotations_fold0.json coco
# !mv train_annotations_fold1.json coco
# !mv train_annotations_fold2.json coco
# !mv train_annotations_fold3.json coco
# !mv train_annotations_fold4.json coco

# !mv val_annotations_fold0.json coco
# !mv val_annotations_fold1.json coco
# !mv val_annotations_fold2.json coco
# !mv val_annotations_fold3.json coco
# !mv val_annotations_fold4.json coco